
# Great Expectations (GX) Setup Notebook

Dieses Notebook dient zur Konfiguration und Definition der Datenvalidierung mit Hilfe der Bibliothek Great Expectations. Die Ergebnisse des Notebooks (Validation_Definition, Checkpoint) werden für die Simulation zur automatisierten Validierung weiterverwendet

## Voraussetzungen
- Abhängigkeiten aus requirements.txt installiert
- Datenbanksystem installiert (siehe README)

## Inhalt
1. **Setup und Datenbankverbindung**  
2. **Definition der Datenvalidierung**  
3. **Definition des Checkpoints**  
4. **Test-Validierung** 

### 1. Setup und Datenbankverbindung

#### Imports

In [1]:
from datetime import datetime, timedelta

# Great Expectations Funktionen importieren
import great_expectations as gx
from great_expectations.checkpoint import UpdateDataDocsAction

# Konstanten für die Datenbankverbindung importieren
from constants import (
    DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME,
    DATA_SOURCE, DATA_ASSET, DB_TABLE, BATCH_DEFINITION,
    EXPECTATION_SUITE, VALIDATION_DEFINITION, CHECKPOINT_NAME
)

#### Kontext laden

In [2]:
# GX-Verzeichnis mit allen relevanten Dateien erstellen
context = gx.get_context(mode="file", project_root_dir="../")

#### Connection-String für Datenbankverbindung generieren

In [3]:
db_connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

#### Datenquelle hinzufügen

In [4]:
data_source = context.data_sources.add_or_update_postgres(
    name = DATA_SOURCE, 
    connection_string = db_connection_string
)

#### Daten-Asset (Tabelle) hinzufügen

In [5]:
if not data_source.assets:
    # Asset existiert noch nicht – daher wird es hinzugefügt
    data_asset = data_source.add_table_asset(table_name=DB_TABLE, name=DATA_ASSET)    
else:
    # Asset existiert bereits
    data_asset = data_source.assets[DATA_ASSET]

#### Batch-Definition hinzufügen

In [6]:
if not data_asset.batch_definitions:
   # Batch-Definition existiert noch nicht – daher wird es hinzugefügt
   batch_definition = data_asset.add_batch_definition_daily(name=BATCH_DEFINITION, column="timestamp")
else:
   # Batch existiert bereits
   batch_definition = data_asset.get_batch_definition(name=BATCH_DEFINITION)

#### Verbindung testen

In [7]:
# Batch laden und die ersten 10 Datensätze ausgeben
daily_batch = batch_definition.get_batch()
daily_batch.head()

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 18.62it/s]


      id                  timestamp sensor_id         location  temperature  \
0  35226 2025-02-28 19:47:30.783011    WS_004           Lindau     9.576649   
1  35227 2025-02-28 19:47:30.856631    WS_001       Ravensburg    12.972763   
2  35228 2025-02-28 19:47:30.940783    WS_002              Ulm    13.592184   
3  35229 2025-02-28 19:47:31.009232    WS_004           Lindau    11.562181   
4  35230 2025-02-28 19:47:31.073529    WS_003  Friedrichshafen    12.536777   

    humidity  wind_speed  
0  82.912090    3.321638  
1  79.155069    3.422189  
2  76.563200    4.719480  
3  63.918741    2.113289  
4  69.299682    2.881031  

### Definition der Datenvalidierung

#### Expectation Suite hinzufügen

In [8]:
gx_suite = context.suites.add_or_update(gx.ExpectationSuite(name=EXPECTATION_SUITE))

#### Expectations definieren

In [9]:
# Validierung von NULL-Werten
exp_not_null_timestamp = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="timestamp", description="Stellt sicher, dass das Zeitstempel-Feld nicht NULL ist."
)
exp_not_null_sensorId = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="sensor_id", description="Stellt sicher, dass das Sensor-ID-Feld nicht NULL ist."
)
exp_not_null_location = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="location", description="Stellt sicher, dass das Standort-Feld nicht NULL ist."
)
exp_not_null_temperature = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="temperature", description="Stellt sicher, dass das Temperatur-Feld nicht NULL ist."
)
exp_not_null_humidity = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="humidity", description="Stellt sicher, dass das Luftfeuchtigkeits-Feld nicht NULL ist."
)
exp_not_null_windSpeed = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="wind_speed", description="Stellt sicher, dass das Windgeschwindigkeits-Feld nicht NULL ist."
)

# Validierung von Eindeutigkeit (Prüfung auf doppelte Datensätze)
exp_unique_measurement = gx.expectations.ExpectCompoundColumnsToBeUnique(
    column_list=["timestamp", "sensor_id", "location"],
    description="Stellt sicher, dass keine Messung redundant vorliegt.",
    
)

# Berechnung des Zeitfensters für die Aktualitätsprüfung
min_timestamp = datetime.now() - timedelta(days=1)  # 24 Stunden zurück

# Validierung der Aktualität
exp_timestamp = gx.expectations.ExpectColumnValuesToBeBetween(
    column="timestamp",
    min_value=min_timestamp,
    description="Stellt sicher, dass der Zeitstempel innerhalb der letzten 24 Stunden liegt."
)

# Validierung von Wertebereichen
exp_temperature = gx.expectations.ExpectColumnValuesToBeBetween(
    column="temperature", min_value=-20, max_value=50,
    description="Stellt sicher, dass die Temperatur zwischen -20 und 50 Grad liegt."
)
exp_humidity = gx.expectations.ExpectColumnValuesToBeBetween(
    column="humidity", min_value=0, max_value=100,
    description="Stellt sicher, dass die Luftfeuchtigkeit zwischen 0 und 100 Prozent liegt."
)
exp_wind_speed = gx.expectations.ExpectColumnValuesToBeBetween(
    column="wind_speed", min_value=0, max_value=50,
    description="Stellt sicher, dass die Windgeschwindigkeit zwischen 0 und 50 km/h liegt."
)

# SQL-Abfrage für Genauigkeitsvalidierung definieren (am Beispiel temperature)
my_query = """
    SELECT
        *
    FROM
        {batch}
    WHERE
        LENGTH(SPLIT_PART(CAST(temperature AS TEXT), '.', 2)) < 1
    """

# Validierung der Genauigkeit (am Beispiel der Spalte temperature)
exp_temperature_precision = gx.expectations.UnexpectedRowsExpectation(
    unexpected_rows_query=my_query,
    description="Stellt sicher, dass die Temperaturwerte mindestens eine Dezimalstelle enthalten."
)

# Validierung der Konsistenz
exp_sensor_id_consistency = gx.expectations.ExpectColumnValuesToBeInSet(
    column="sensor_id", value_set=["WS_001", "WS_002", "WS_003", "WS_004", "WS_005"],
    description="Stellt sicher, dass die Sensor-ID einem definierten Wert entspricht."
)

#### Expectations zum GX-Kontext hinzufügen

In [10]:
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_timestamp)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_sensorId)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_location)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_temperature)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_humidity)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_not_null_windSpeed)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_unique_measurement)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_timestamp)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_temperature)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_humidity)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_wind_speed)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_temperature_precision)
context.suites.get(name=EXPECTATION_SUITE).add_expectation(exp_sensor_id_consistency)



ExpectColumnValuesToBeInSet(id='df3df562-6044-4798-98cd-457f6f2ef133', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description='Stellt sicher, dass die Sensor-ID einem definierten Wert entspricht.', catch_exceptions=True, rendered_content=None, windows=None, batch_id=None, column='sensor_id', mostly=1, row_condition=None, condition_parser=None, value_set=['WS_001', 'WS_002', 'WS_003', 'WS_004', 'WS_005'])

#### Validation-Definition zum Kontext hinzufügen

In [11]:
validation_definition = gx.ValidationDefinition(
    data=batch_definition, 
    suite=context.suites.get(name=EXPECTATION_SUITE), 
    name=VALIDATION_DEFINITION
)
validation_definition = context.validation_definitions.add_or_update(validation_definition)

### Definition des Checkpoints

In [12]:
action_list = [    
    # Diese Aktion aktualisiert die Data Docs statische Website mit den Validierungsergebnissen,
    # nachdem der Checkpoint ausgeführt wurde.
    UpdateDataDocsAction(
        name="update_all_data_docs",
    ),
]

# Checkpoint erstellen
checkpoint = gx.Checkpoint(
    name=CHECKPOINT_NAME,
    validation_definitions=[context.validation_definitions.get(name=VALIDATION_DEFINITION)],
    actions=action_list,
    result_format={"result_format": "COMPLETE"},
)

# Checkpoint zum Kontext hinzufügen
context.checkpoints.add_or_update(checkpoint)

Checkpoint(name='weather_data_validation_checkpoint', validation_definitions=[ValidationDefinition(name='weather_data_validation_definition', data=BatchDefinition(id=UUID('34eaf145-6208-4671-b051-488272f958fe'), name='last_day_batch', partitioner=ColumnPartitionerDaily(column_name='timestamp', sort_ascending=True, method_name='partition_on_year_and_month_and_day')), suite={
  "name": "weather_data_expectation_suite",
  "id": "63d263bc-0207-4380-a604-8add5070ed0c",
  "expectations": [
    {
      "type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "timestamp"
      },
      "meta": {},
      "id": "9b0e5763-4ff3-477e-82b9-753a5cec2507",
      "description": "Stellt sicher, dass das Zeitstempel-Feld nicht NULL ist."
    },
    {
      "type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "sensor_id"
      },
      "meta": {},
      "id": "41b497cd-2cc4-4573-8ec1-9182fdfc9ba6",
      "description": "Stellt sicher, dass das Sensor

### Test-Validierung

In [13]:
# Vorhandenen Checkpoint anhand seines Namens abrufen und ausführen
checkpoint_test = context.checkpoints.get(name=CHECKPOINT_NAME)
checkpoint_test.run()

Calculating Metrics: 100%|██████████| 84/84 [00:00<00:00, 496.17it/s]


CheckpointResult(run_id={"run_name": null, "run_time": "2025-02-28T19:48:37.037653+01:00"}, run_results={ValidationResultIdentifier::weather_data_expectation_suite/__none__/20250228T184837.037653Z/postgres_weather_db-measurement-year_2025-month_2-day_28: {
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "postgres_weather_db-measurement-year_2025-month_2-day_28",
          "column": "timestamp"
        },
        "meta": {},
        "id": "9b0e5763-4ff3-477e-82b9-753a5cec2507",
        "description": "Stellt sicher, dass das Zeitstempel-Feld nicht NULL ist."
      },
      "result": {
        "element_count": 404,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": [],
        "unexpected_list": [],
        "unexpected_index_query": "SELECT timestamp \nFROM 